In [ ]:
!kill -9 -1 #reiniciando la maquina virtual

In [32]:
#!pip list
!nvcc --version  #Version de CUDA en la maquina virtual

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [2]:
import tensorflow as tf
import timeit               #para medir tiempos
import numpy as np
import pandas as pd 
import os
import time
import sys

In [33]:
print("Tensorflow Version: ", tf.__version__)
print("Dispositivos disponibles para entrenar: ", tf.config.list_physical_devices())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Encontrada la GPU: {}'.format(device_name))

Tensorflow Version:  2.7.0
Dispositivos disponibles para entrenar:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


SystemError: ignored

In [34]:
def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)   

In [35]:
cpu()  #ejecutamos entrenamiento con CPU
gpu()  #ejecutamos entrenamiento con GPU
# Run the op several times.
print('TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes '
      '(batch x height x width x channel). suma de 10 epochs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes (batch x height x width x channel). suma de 10 epochs.
CPU (s):
3.4569260600001144
GPU (s):
3.4462130399997477
GPU speedup over CPU: 1x


In [36]:
#tf.device('/gpu:0') #activando la CPU
tf.device('/device:GPU:0') #activando la GPU 

In [37]:
fileDL= tf.keras.utils.get_file('codigodavinci.txt','https://raw.githubusercontent.com/DerUwU/Deep-Learning/main/Trabajos/Redes%20Neuronales%20Recurrentes/Datasets/codigodavinci.txt')
texto = open(fileDL, 'rb').read().decode(encoding='cp1252')


In [38]:
import re
from unicodedata import normalize
#pasa todo a minuscula
texto     = texto.lower()
#reemplazar tildes por letras similares sin tildes
transfor  = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
texto     = normalize('NFKC', normalize('NFKD', texto).translate(transfor))
#quitar saltos de linea
texto      = texto.strip()
texto      = re.sub('\r|\n', ' ',texto)
#quitar espacios dobles
texto      = re.sub(' +', ' ', texto)
#quitando caracteres especiales
texto = re.sub(r"[^a-zA-Z0-9]+"," ",texto)
print(texto)

el codigo da vinci dan brown para blythe una vez mas mas que nunca agradecimientos en primer lugar le doy las gracias a mi amigo y editor jason kaufman por involucrarse tanto en este proyecto y por entender plenamente de que trata este libro gracias tambien a la incomparable heide lange campeona incansable de el codigo da vinci extraordinaria agente y amiga de verdad no tengo palabras para expresar la gratitud que siento por el excepcional equipo de doubleday por su generosidad su fe y su inestimable ayuda gracias especialmente a bill thomas y a steve rubin que creyeron en este libro desde el principio gracias tambien al primer grupo de defensores de la obra en sus etapas iniciales encabezado por michael palgon suzanne herz janelle moburg jackie everly y adrienne sparks ademas de a los muy buenos profesionales del equipo de ventas de doubleday y a michael windsor por la atractiva cubierta de la edicion norteamericana por su desinteresada ayuda en la investigacion necesaria para la prep

In [39]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:870915 caracteres
el texto esta compuesto de estos :37 caracteres
[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [40]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones

#for char,_ in zip(char2idx, range(len(vocab))):
    #print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'el codigo da vinci dan brown para blythe una vez mas mas que nunca agradecimientos en primer lugar l'
array([15, 22,  0, 13, 25, 14, 19, 17, 25,  0, 14, 11,  0, 32, 19, 24, 13,
       19,  0, 14, 11, 24,  0, 12, 28, 25, 33, 24,  0, 26, 11, 28, 11,  0,
       12, 22, 35, 30, 18, 15,  0, 31, 24, 11,  0, 32, 15, 36,  0, 23, 11,
       29,  0, 23, 11, 29,  0, 27, 31, 15,  0, 24, 31, 24, 13, 11,  0, 11,
       17, 28, 11, 14, 15, 13, 19, 23, 19, 15, 24, 30, 25, 29,  0, 15, 24,
        0, 26, 28, 19, 23, 15, 28,  0, 22, 31, 17, 11, 28,  0, 22])


In [41]:
rows=[]
columns=['num','vocab']
for i, voc in enumerate(vocab):
  #print(i,'-->', voc)
  rows.append([i,voc])
df= pd.DataFrame(columns=['num','vocab'],data=rows)
df.head(10)
df.to_csv('data_vocab.csv',index=False)

In [42]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=50
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'el codigo da vinci dan brown para blythe una vez ma'
's mas que nunca agradecimientos en primer lugar le '
'doy las gracias a mi amigo y editor jason kaufman p'
'or involucrarse tanto en este proyecto y por entend'
'er plenamente de que trata este libro gracias tambi'
'en a la incomparable heide lange campeona incansabl'
'e de el codigo da vinci extraordinaria agente y ami'
'ga de verdad no tengo palabras para expresar la gra'
'titud que siento por el excepcional equipo de doubl'
'eday por su generosidad su fe y su inestimable ayud'


In [43]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'el codigo da vinci dan brown para blythe una vez m'
Target data:  'l codigo da vinci dan brown para blythe una vez ma'


In [44]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((50,), (50,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 50), (64, 50)), types: (tf.int64, tf.int64)>


In [45]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam', #
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (64, None, 256)           9472      
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_3 (Dense)             (64, None, 37)            37925     
                                                                 
Total params: 5,294,373
Trainable params: 5,294,373
Non-trainable params: 0
_________________________________________________________________


In [46]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [47]:
checkpoint_dir='/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpointsv4/'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')


In [84]:
EPOCHS=300
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback]) #0.3

Epoch 1/300


ValueError: ignored

In [48]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpointsv4/cp_0144.ckpt


In [22]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history2=model.fit(dataset, 
                    epochs=500,  
                    verbose=1,
                    callbacks=[cp_callback])

Epoch 1/500


ValueError: ignored

In [49]:
# You can change the directory name
LOG_DIR = 'tb_logs'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-11-25 19:43:55--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  50.7MB/s    in 0.3s    

2021-11-25 19:43:55 (50.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
https://04da-104-196-106-63.ngrok.io


In [50]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=BATCH_SIZE,
                         write_images=True)

In [51]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [52]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.1  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


In [53]:
print(generate_text(model, start_string=u"que onda"))

que onda en si mismo por tanto este ha sido un tiempo de religion hibre de la empresa de taxis y se oia por un altavoz que llevaba paso el dedo por el engarce en los labios cuando era peque a ha hecho bien armado en sus enormes puertas abiertos llave algo en el a o era simplemente un caballero el obispo medito sobre la amatista purpura de la divinidad femenina perdida con repentina empezaron a encenderse algunas emedias que lo retener la mayor parte de su tierra muy seria langdon no dejo de sospechar qu


In [54]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Colab Notebooks/Modelos'
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_ElCodigoDavinci_json.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_ElCodigoDavinci_pesos.hdf5'))
model.save(os.path.join(dir_export,'RNN_ElCodigoDavinci_model.h5'))
print("Modelo salvado en Drive de google")

Modelo salvado en Drive de google


In [55]:
!wget https://github.com/DerUwU/Deep-Learning/blob/main/Trabajos/Redes%20Neuronales%20Recurrentes/HDF5/RNN_ElCodigoDavinci_pesos.hdf5?raw=true \
      -O RNN_ElCodigoDavinci_model.h5

--2021-11-25 19:45:44--  https://github.com/DerUwU/Deep-Learning/blob/main/Trabajos/Redes%20Neuronales%20Recurrentes/HDF5/RNN_ElCodigoDavinci_pesos.hdf5?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/DerUwU/Deep-Learning/raw/main/Trabajos/Redes%20Neuronales%20Recurrentes/HDF5/RNN_ElCodigoDavinci_pesos.hdf5 [following]
--2021-11-25 19:45:44--  https://github.com/DerUwU/Deep-Learning/raw/main/Trabajos/Redes%20Neuronales%20Recurrentes/HDF5/RNN_ElCodigoDavinci_pesos.hdf5
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DerUwU/Deep-Learning/main/Trabajos/Redes%20Neuronales%20Recurrentes/HDF5/RNN_ElCodigoDavinci_pesos.hdf5 [following]
--2021-11-25 19:45:44--  https://raw.githubusercontent.com/DerUwU/Deep-Learning/main/Trabajos/Redes%20Neuronal

In [56]:
!pip install pyprind

In [57]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()

import urllib.request
url_github_Model='https://github.com/DerUwU/Deep-Learning/blob/main/Trabajos/Redes%20Neuronales%20Recurrentes/HDF5/RNN_ElCodigoDavinci_pesos.hdf5?raw=true'
urllib.request.urlretrieve(url_github_Model,
                           'RNN_ElCodigoDavinci_model.h5', 
                           reporthook)

100% | 20 MB | 4.01 MB/s | 5 segundos transcurrido

('RNN_ElCodigoDavinci_model.h5', <http.client.HTTPMessage at 0x7faba4846790>)

In [61]:
new_model = tf.keras.models.load_model('/content/gdrive/MyDrive/Colab Notebooks/Modelos/RNN_ElCodigoDavinci_model.h5')

In [62]:
df2 = pd.read_csv("https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/master/DeepLearning/PLN/recurrent_network_RNN/Modelos/data_vocab.csv")
df2.head()

,num,vocab
0,0,
1,1,0
2,2,1
3,3,2
4,4,3


In [63]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval  = [char2idx[s] for s in start_string]
  input_eval  = tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


In [64]:
print(generate_text(new_model, start_string=u"los fantasmas de "))

los fantasmas de la limusina la policia no parecia ser un hombre experimentaba una emocion que le era familiar cuando intentar adelante y la entrada de la iglesia deteniendose en la penumbra se acerco mas a la mosquitera aquel momento no se estaba comunicando de alguna otra mano trazo una forma si ha sido con algo que llevo a mi abuelo a confiar en ti re y a facilitarle al contemplar aquel techo al final de la postarlo y ahora langdon y sophie 286 hemos veces se llama leigh teabing es un antiguo miembro de la fe
